<h1 align= "center"><b>Problem: Expanding Nebula</b></h1>

You've escaped Commander Lambda's exploding space station along with numerous escape pods full of bunnies. But - oh no! - one of the escape pods has flown into a nearby nebula, causing you to lose track of it. You start monitoring the nebula, but unfortunately, just a moment too late to find where the pod went. However, you do find that the gas of the steadily expanding nebula follows a simple pattern, meaning that you should be able to determine the previous state of the gas and narrow down where you might find the pod.

From the scans of the nebula, you have found that it is very flat and distributed in distinct patches, so you can model it as a 2D grid. You find that the current existence of gas in a cell of the grid is determined exactly by its 4 nearby cells, specifically, (1) that cell, (2) the cell below it, (3) the cell to the right of it, and (4) the cell below and to the right of it. If, in the current state, exactly 1 of those 4 cells in the 2x2 block has gas, then it will also have gas in the next state. Otherwise, the cell will be empty in the next state.

For example, let's say the previous state of the grid (p) was:

    .O..
    ..O.
    ...O
    O...

To see how this grid will change to become the current grid (c) over the next time step, consider the 2x2 blocks of cells around each cell.  Of the 2x2 block of [p[0][0], p[0][1], p[1][0], p[1][1]], only p[0][1] has gas in it, which means this 2x2 block would become cell c[0][0] with gas in the next time step:

    .O -> O
    ..

Likewise, in the next 2x2 block to the right consisting of [p[0][1], p[0][2], p[1][1], p[1][2]], two of the containing cells have gas, so in the next state of the grid, c[0][1] will NOT have gas:

    O. -> .
    .O

Following this pattern to its conclusion, from the previous state p, the current state of the grid c will be:

    O.O
    .O.
    O.O

Note that the resulting output will have 1 fewer row and column, since the bottom and rightmost cells do not have a cell below and to the right of them, respectively.

Write a function solution(g) where g is an array of array of bools saying whether there is gas in each cell (the current scan of the nebula), and return an int with the number of possible previous states that could have resulted in that grid after 1 time step.  For instance, if the function were given the current state c above, it would deduce that the possible previous states were p (given above) as well as its horizontal and vertical reflections, and would return 4. The width of the grid will be between 3 and 50 inclusive, and the height of the grid will be between 3 and 9 inclusive.  The answer will always be less than one billion (10^9).


<h3 align= "center"><b>Test Cases</b></h3>

```

Input:
    solution.solution([[True, True, False, True, False, True, False, True, True, False], [True, True, False, False, False, False, True, True, True, False], [True, True, False, False, False, False, False, False, False, True], [False, True, False, False, False, False, True, True, False, False]])
Output:
    11567


Input:
    solution.solution([[True, False, True], [False, True, False], [True, False, True]])
Output:
    4

Input:
    solution.solution([[True, False, True, False, False, True, True, True], [True, False, True, False, False, False, True, False], [True, True, True, False, False, False, True, False], [True, False, True, False, False, False, True, False], [True, False, True, False, False, True, True, True]])
Output:
    254

```

<h3 align= "center"><b>The idea</b></h3>

We do dynamic programming using all the different possible columns for the previous state. We represent each column as a number between 0 and 2^(height+1) that when written in binary will have 0 = no gas, 1 = gas. We keep a dictionary with all possible columns, each of them initialized with value 1. Then we can traverse the matrix by column , do some bitwise operations to compute the resulting column given each possible current and previous column, and if the result matches the one in the input matrix we add the corresponding value of the dictionary to the current column. That number will represent the number of solutions of the submatrix (from right to left up to that column position) that end with that column, so after all iterations we sum all the values for all possible columns and return that. This accepted solution will then run in O(width * 2^(height+1)), note that width * 2^(height+1) <= 500 * 1024^2 which is less than 10^7 but still a big number, so I do wonder if a faster solution is possible...

<h3 align= "center"><b>The code</b></h3>

In [3]:
def solution(g):
    
    height = len(g)
    width = len(g[0])
    
    expected_columns = [0 for _ in range(width)]
    # compute expected_columns as in input g, written in binary and reversed horizontally:
    for h in range(height):
        for c in range(width):
            expected_columns[c] = 2*expected_columns[c] + g[h][width-1-c]
            
    possible_columns = [1 for c in range(2**(height+1))]
    
    # compute the new column given two columns c1 and c2
    def compute(c1,c2):
        c11 = c1%(2**height)
        c12 = c1>>1
        c21 = c2%(2**height)
        c22 = c2>>1
        there_s_a_1 = c11 | c12 | c21 | c22
        there_s_more_than_one_1 = (c11 & c12) | (c11 & c21) | (c11 & c22) | (c12 & c21) | (c12 & c22) | (c21 & c22)
        return there_s_a_1 & ~there_s_more_than_one_1
        
    
    for i in range(width):
        new_possible_columns = [0 for c in range(2**(height+1))]
        for col1,val in enumerate(possible_columns):
            for col2 in range(2**(height+1)):
                if compute(col1,col2) == expected_columns[i]:
                    new_possible_columns[col2] += val
        possible_columns = new_possible_columns
    
    return sum( v for v in new_possible_columns )

In [4]:
print (solution([[True, True, False, True, False, True, False, True, True, False], [True, True, False, False, False, False, True, True, True, False], [True, True, False, False, False, False, False, False, False, True], [False, True, False, False, False, False, True, True, False, False]]))

11567


In [5]:
print(solution([[True, False, True], [False, True, False], [True, False, True]]))

4


In [6]:
print(solution([[True, False, True, False, False, True, True, True], [True, False, True, False, False, False, True, False], [True, True, True, False, False, False, True, False], [True, False, True, False, False, False, True, False], [True, False, True, False, False, True, True, True]]))

254
